In [ ]:
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

import cv2
import glob

import pandas as pd

### Load Model

In [ ]:
weights_path = "path/to/weights/efficientnetb0_weights-notop.h5"
model = EfficientNetB0(weights=weights_path,
                       include_top=False,
                       input_shape=(None, None, 3),
                       pooling="avg")

#### Example 1. Extract features for an image

In [ ]:
# Load image
img_path = "path/to/image/img"
img = cv2.imread(img_path)

# Extract 1280-dimensional features from an image
features = model.predict(img)

#### Example 2. Extract features for a folder of images

In [ ]:
# Path to image folder
folder_path = "path/to/image/folder/"

# Create efficient data generators
data_generator = ImageGenerator().flow_from_directory(folder_path, class_mode=None)
ds_data_generator = tf.data.Dataset.from_generator(lambda: data_generator, output_types=(tf.float32))
ds_data_generator = ds_data_generator.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
steps_to_predict = ceil(len(data_generator) / batch_size)

# Extract 1280-dimensional features from each image
all_features = model.predict(ds_data_generator, steps=steps_to_predict)

# Save features. Each row is a 1280-dim feature vector corresponding to an image.
df_features = pd.DataFrame(all_features)
df_features.to_hdf("path/to/save/features.h5")